<a href="https://colab.research.google.com/github/btg1631/study_data_analytics/blob/main/docs/quests/CDAs/CDA_mixed_unVar2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 한글 폰트 설치 및 시각화

In [19]:
# 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab 경우 Reuntime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 32 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 3s (3,702 kB/s)
Selecting previously unselected package fonts-nanum.
(Reading database ... 121747 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/humor-sans: caching, new cache contents: 1 fonts, 0 dirs
/usr

In [12]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

## load dataset file

In [2]:
import pandas as pd
df_fruits = pd.read_csv('fruitstore_saleshistory.csv')
df_fruits[:2]

,공급일자,요일,공급월,공급주차,회원번호,조합원상태,물품대분류,물품중분류,물품소분류,물품명,구매수량,주소-구,주소-동,성별,연령,연령대,구매금액,구매매장,반품_원거래일자,구매시각
0,2018-01-02,화,1,1,272369856,정상회원,과실,과일,사과,사과/유(1.5kg),1.0,수지구,풍덕천동,여,45.0,40대,22207,매장C,NaN,10:04
1,2018-01-02,화,1,1,1506656256,정상회원,채소,버섯,느타리버섯,느타리버섯(300g),1.0,수지구,풍덕천동,여,36.0,30대이하,4977,매장C,NaN,10:05


In [3]:
df_fruits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705571 entries, 0 to 705570
Data columns (total 20 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   공급일자      705571 non-null  object 
 1   요일        705571 non-null  object 
 2   공급월       705571 non-null  int64  
 3   공급주차      705571 non-null  int64  
 4   회원번호      705571 non-null  int64  
 5   조합원상태     705571 non-null  object 
 6   물품대분류     705571 non-null  object 
 7   물품중분류     705571 non-null  object 
 8   물품소분류     705571 non-null  object 
 9   물품명       705571 non-null  object 
 10  구매수량      705571 non-null  float64
 11  주소-구      705571 non-null  object 
 12  주소-동      705571 non-null  object 
 13  성별        705566 non-null  object 
 14  연령        705538 non-null  float64
 15  연령대       705538 non-null  object 
 16  구매금액      705571 non-null  int64  
 17  구매매장      705571 non-null  object 
 18  반품_원거래일자  11234 non-null   float64
 19  구매시각      705571 non-null  object 
dtypes: f

In [4]:
df_fruits.isnull().sum()

공급일자             0
요일               0
공급월              0
공급주차             0
회원번호             0
조합원상태            0
물품대분류            0
물품중분류            0
물품소분류            0
물품명              0
구매수량             0
주소-구             0
주소-동             0
성별               5
연령              33
연령대             33
구매금액             0
구매매장             0
반품_원거래일자    694337
구매시각             0
dtype: int64

## 범주형 / 연속형이고 이분산인 경우

- 요일별 고객의 구매 금액 분산이 이분산을 보이는지?
 + 요일(범주형)에 따른 고객의 구매 금액(연속형) 분산을 비교하여, 특정 요일에 구매 금액의 변동성이 큰지 분석합니다.


### 분석 내용
 + 귀무 가설 : 요일별 구매금액의 차이가 없다.
 + 대립 가설 : 요일별 구매금액의 차이가 있다.

In [5]:
import scipy.stats as stats

# 구매금액(연속형) 정규성 검증  ->  비정규 분포
result = stats.anderson(df_fruits['구매금액'].dropna())
print('Test Statistic: %.3f' % result.statistic)
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    if result.statistic < cv:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따름 (검정 통계량 %.3f < 임계값 %.3f)' % (sl, result.statistic, cv))
    else:
        print('%.1f%% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 %.3f >= 임계값 %.3f)' % (sl, result.statistic, cv))

Test Statistic: 80887.237
15.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.576)
10.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.656)
5.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.787)
2.5% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 0.918)
1.0% 수준에서 데이터는 정규 분포를 따르지 않음 (검정 통계량 80887.237 >= 임계값 1.092)


In [6]:
week_list = df_fruits['요일'].value_counts().index.to_list()
week_list

['월', '화', '금', '수', '목', '토', '일']

In [7]:
series_list =  list()
for week in week_list:
  condition = f'요일 == "{week}"'
  series_amount = df_fruits.query(condition)['구매금액']
  series_list.append(series_amount)
series_list[6]

22160      6509
22161      3637
22162      2489
22163      4977
22164     10912
          ...  
678822    12635
678823    26610
678824     7658
678825    10338
678826     2106
Name: 구매금액, Length: 54077, dtype: int64

In [8]:
stats.kruskal(series_list[0], series_list[1], series_list[2], series_list[3], series_list[4], series_list[5], series_list[6])
# statistic=105.68, pvalue=1.6278456577231045e-20

KruskalResult(statistic=105.68763451021304, pvalue=1.6278456577231045e-20)

In [9]:
round(1.6278456577231045e-20)

0

#### 분석 결과
- 통계 결론 : 귀무가설 기각
- 사실 결론 : 요일별 구매금액의 차이가 있다.

In [40]:
grouped = df_fruits.groupby('요일')['구매금액'].mean()
grouped.sort_values()

요일
월    10542.528917
목    10857.069337
일    10939.249736
화    11002.352378
토    11046.589839
수    11062.047300
금    11064.344340
Name: 구매금액, dtype: float64